In [1]:

from pyflink.common import Row
from pyflink.table import (EnvironmentSettings, TableEnvironment, TableDescriptor, Schema,
                           DataTypes, FormatDescriptor)
from pyflink.table.expressions import lit, col
from pyflink.table.udf import udtf

In [2]:

word_count_data = ["To be, or not to be,--that is the question:--",
                   "Whether 'tis nobler in the mind to suffer",
                   "The slings and arrows of outrageous fortune",
                   "Or to take arms against a sea of troubles,",
                   "And by opposing end them?--To die,--to sleep,--",
                   "No more; and by a sleep to say we end",
                   "The heartache, and the thousand natural shocks",
                   "That flesh is heir to,--'tis a consummation",
                   "Devoutly to be wish'd. To die,--to sleep;--",
                   "To sleep! perchance to dream:--ay, there's the rub;",
                   "For in that sleep of death what dreams may come,",
                   "When we have shuffled off this mortal coil,",
                   "Must give us pause: there's the respect",
                   "That makes calamity of so long life;",
                   "For who would bear the whips and scorns of time,",
                   "The oppressor's wrong, the proud man's contumely,",
                   "The pangs of despis'd love, the law's delay,",
                   "The insolence of office, and the spurns",
                   "That patient merit of the unworthy takes,",
                   "When he himself might his quietus make",
                   "With a bare bodkin? who would these fardels bear,",
                   "To grunt and sweat under a weary life,",
                   "But that the dread of something after death,--",
                   "The undiscover'd country, from whose bourn",
                   "No traveller returns,--puzzles the will,",
                   "And makes us rather bear those ills we have",
                   "Than fly to others that we know not of?",
                   "Thus conscience does make cowards of us all;",
                   "And thus the native hue of resolution",
                   "Is sicklied o'er with the pale cast of thought;",
                   "And enterprises of great pith and moment,",
                   "With this regard, their currents turn awry,",
                   "And lose the name of action.--Soft you now!",
                   "The fair Ophelia!--Nymph, in thy orisons",
                   "Be all my sins remember'd."]

In [3]:
t_env = TableEnvironment.create(EnvironmentSettings.in_streaming_mode())
# write all the data to one file
t_env.get_config().set("parallelism.default", "1")

tab = t_env.from_elements(map(lambda i: (i,), word_count_data),
                                  DataTypes.ROW([DataTypes.FIELD('line', DataTypes.STRING())]))

t_env.create_temporary_table(
            'sink',
            TableDescriptor.for_connector('print')
                           .schema(Schema.new_builder()
                                   .column('word', DataTypes.STRING())
                                   .column('count', DataTypes.BIGINT())
                                   .build())
                           .build())



In [4]:
@udtf(result_types=[DataTypes.STRING()])
def split(line: Row):
    for s in line[0].split():
        yield Row(s)

In [5]:
# compute word count
tab.flat_map(split).alias('word') \
   .group_by(col('word')) \
   .select(col('word'), lit(1).count) \
   .execute_insert('sink') \
   .wait()

+I[To, 1]
+I[be,, 1]
+I[or, 1]
+I[not, 1]
+I[to, 1]
+I[be,--that, 1]
+I[is, 1]
+I[the, 1]
+I[question:--, 1]
+I[Whether, 1]
+I['tis, 1]
+I[nobler, 1]
+I[in, 1]
-U[the, 1]
+U[the, 2]
+I[mind, 1]
-U[to, 1]
+U[to, 2]
+I[suffer, 1]
+I[The, 1]
+I[slings, 1]
+I[and, 1]
+I[arrows, 1]
+I[of, 1]
+I[outrageous, 1]
+I[fortune, 1]
+I[Or, 1]
-U[to, 2]
+U[to, 3]
+I[take, 1]
+I[arms, 1]
+I[against, 1]
+I[a, 1]
+I[sea, 1]
-U[of, 1]
+U[of, 2]
+I[troubles,, 1]
+I[And, 1]
+I[by, 1]
+I[opposing, 1]
+I[end, 1]
+I[them?--To, 1]
+I[die,--to, 1]
+I[sleep,--, 1]
+I[No, 1]
+I[more;, 1]
-U[and, 1]
+U[and, 2]
-U[by, 1]
+U[by, 2]
-U[a, 1]
+U[a, 2]
+I[sleep, 1]
-U[to, 3]
+U[to, 4]
+I[say, 1]
+I[we, 1]
-U[end, 1]
+U[end, 2]
-U[The, 1]
+U[The, 2]
+I[heartache,, 1]
-U[and, 2]
+U[and, 3]
-U[the, 2]
+U[the, 3]
+I[thousand, 1]
+I[natural, 1]
+I[shocks, 1]
+I[That, 1]
+I[flesh, 1]
-U[is, 1]
+U[is, 2]
+I[heir, 1]
+I[to,--'tis, 1]
-U[a, 2]
+U[a, 3]
+I[consummation, 1]
+I[Devoutly, 1]
-U[to, 4]
+U[to, 5]
+I[be, 1]
+I[wish'd.